In [1]:
import os
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

import get_env
env = get_env.get_remote_env()
t_env = StreamTableEnvironment.create(env)

2025-12-03T16:36:47.833002Z main ERROR Reconfiguration failed: No configuration found for '567d299b' at 'null' in 'null'
2025-12-03T16:36:48.951226Z Thread-3 ERROR Reconfiguration failed: No configuration found for '761badbc' at 'null' in 'null'


In [2]:

t_env.execute_sql("DROP TABLE IF EXISTS test_out")

t_env.execute_sql("""
CREATE TABLE test_out (
  k STRING,
  v STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'test',
  'properties.bootstrap.servers' = 'broker:9092',
  'key.format' = 'raw',
  'key.fields' = 'k',
  'value.format' = 'raw',
  'value.fields-include' = 'EXCEPT_KEY'
)
""")

# write 3 records and finish
result = t_env.execute_sql("""
INSERT INTO test_out
SELECT * FROM (
  VALUES
    ('key-1', 'value-1'),
    ('key-2', 'value-2'),
    ('key-3', 'value-3')
) AS T(k, v)
""")

In [4]:
job_id = result.get_job_client().get_job_id()
print ("job", job_id)

status = result.get_job_client().get_job_status().result()
 
# keep checking until FINISHED
print("status:", status, str(status))

job 6cfad4860c02e2aee74f0ded373bfc42
status: JobStatus.FINISHED JobStatus.FINISHED
